# Show some basics and options for the retrieval
This notebooks reads a sample spectra and illustrates the retrieval of the PSD similar to Barret et al. (2019) and some sensitivities for the particle properties

In [ ]:
from PSDretrieval import processRadar as pR
from PSDretrieval import plotting as pl
from PSDretrieval import scattering as sc
from PSDretrieval import retrievalUtils as rU
import snowScatt
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
#define time
date = "20190113"
time = "06:18:04"
hcenter=1600 #[m]
#load sample data (this is a window of time and height, which can define by additional input parameters)
SpecWindow  = pR.loadSpectra(date=date,time=time,hcenter=hcenter)
#load the single spectra
SpecSingle  = pR.selectSingleTimeHeight(SpecWindow)
#load detected peaks from processed file
PeaksWindow  = pR.loadPeaks()

## plot the sample spectrum

In [ ]:
fig,ax = plt.subplots(nrows=1,ncols=1)
ax = pl.plotObsSpectra(SpecSingle,ax)

## get the vertical wind estimate

In [ ]:
#add vertical wind information to 
SpecWindow = pR.addVerticalWindToSpecWindow(SpecWindow,PeaksWindow)
#extract a single spectra from the time-height window (no with the W-information)
SpecSingle  = pR.selectSingleTimeHeight(SpecWindow)
#get again the - now shifted - single spectra from the SpecWindow 
SpecSingleWshifted  = pR.shiftSpectra(SpecSingle)

## plot W-shifted spectra

In [ ]:
fig,ax = plt.subplots(nrows=1,ncols=1)
ax = pl.plotObsSpectra(SpecSingleWshifted,ax)

## cut noisy part and plot spectra again

In [ ]:
fig,ax = plt.subplots(nrows=1,ncols=1)
#cut spectra
SpecWindow = pR.cutLowZe(SpecWindow,zeThreshold=-20)
SpecSingleWshiftedCut = pR.cutLowZe(SpecSingleWshifted,zeThreshold=-20)
ax = pl.plotObsSpectra(SpecSingleWshiftedCut,ax)

## plot single-spectra DV vs. DWR

In [ ]:
fig2,axes2 = plt.subplots(nrows=1,ncols=2,figsize=(20,10))
__ = pl.plotSDWRvsDVobs(SpecSingleWshiftedCut,axes2)

## plot average DV vs. DWR from time-height window

In [ ]:
fig2,axes2 = plt.subplots(nrows=1,ncols=2,figsize=(20,10))
__ = pl.plotSDWRvsDVobs(SpecWindow,axes2)

## how do the snowscatt models fit to the observed DV-DWR relation

### mixcoldend with all riming degrees

In [ ]:
fig2,axes2 = plt.subplots(nrows=1,ncols=2,figsize=(20,10))
axes2 = pl.plotSDWRvsDVobs(SpecWindow,axes2)
#get names of all particle types
allParticleTypes        = [*snowScatt.snowLibrary._fileList.keys()]
#select an aggregate type
aggType = 'vonTerzi_mixcoldend'
#get a list of all particle-type names from this aggregate type with different riming degrees
allRimDegr = [k for k in allParticleTypes if aggType in k]
for pType in allRimDegr:
    #get spectral-resolved particle properties
    Zx, Zk, Zw, Dmax, K2, vel = sc.model3fOne(pType)
    #calculate spectral DWRs
    DWRxk = Zx-Zk; DWRkw = Zk-Zw
    axes2 = pl.plotSDWRvsDVmodel(vel,DWRxk,DWRkw,axes2,pType)
axes2[0].legend()

### all unrimed vonTerzis'

In [ ]:
fig2,axes2 = plt.subplots(nrows=1,ncols=2,figsize=(20,10))
allParticleTypes        = [*snowScatt.snowLibrary._fileList.keys()]
axes2 = pl.plotSDWRvsDVobs(SpecWindow,axes2)
allUnrimVT = [k for k in allParticleTypes if ("vonTerzi_" in k and not "rimed" in k)]
for pType in allUnrimVT:
    #get spectral-resolved particle properties
    Zx, Zk, Zw, Dmax, K2, vel = sc.model3fOne(pType)
    #calculate spectral DWRs
    DWRxk = Zx-Zk; DWRkw = Zk-Zw
    axes2 = pl.plotSDWRvsDVmodel(vel,DWRxk,DWRkw,axes2,pType)
axes2[0].legend()

In [ ]:
### all Leinonens' and Ori

In [ ]:
fig2,axes2 = plt.subplots(nrows=1,ncols=2,figsize=(20,10))
allParticleTypes        = [*snowScatt.snowLibrary._fileList.keys()]
axes2 = pl.plotSDWRvsDVobs(SpecWindow,axes2)
allUnrimVT = [k for k in allParticleTypes if ("Leinonen" in k or "Ori" in k)]
for pType in allUnrimVT:
    #get spectral-resolved particle properties
    Zx, Zk, Zw, Dmax, K2, vel = sc.model3fOne(pType)
    #calculate spectral DWRs
    DWRxk = Zx-Zk; DWRkw = Zk-Zw
    axes2 = pl.plotSDWRvsDVmodel(vel,DWRxk,DWRkw,axes2,pType)
axes2[0].legend()

In [ ]:
### find best fitting particle type

In [ ]:
#find best fitting particle type
#ParticleTypeList = [*snowScatt.snowLibrary._fileList.keys()] #read https://www.python.org/dev/peps/pep-0448/ for the [*...] formalism
ParticleTypeList = [k for k in allParticleTypes if 'vonTerzi_mixcoldend' in k]
#ParticleTypeList = ["vonTerzi_mixcoldend","vonTerzi_mixcoldend_rimed05"]
[bestPartType,orderedListPartType] = rU.findBestFittingPartType(ParticleTypeList,SpecWindow)